In [1]:
%load_ext autoreload
%autoreload 2

import torch
from mnist import *
from ed import *

In [88]:
list(range(4-1,-1,-1))

[3, 2, 1, 0]

In [85]:
idx = torch.ones([10]).multinomial(num_samples=14, replacement=True)
idx
torch.eye(src_shape[0])[np.random.choice(src_shape[0], W.shape[0])]

tensor([9, 8, 0, 4, 6, 9, 4, 2, 3, 5, 9, 4, 3, 2])

In [116]:
model.linops[0].weight.shape

torch.Size([10, 784])

In [3]:
args = parse_txt("--arch fc --layers 0 --batch 16")
print(args)
model, dl_train, dl_val, device, aname, save_path = get_model(args)

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum, nesterov=args.nesterov, weight_decay=args.weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, (len(dl_train))*args.epochs)


Namespace(act='relu', arch='fc', aug_lr=0.001, augment=True, batch_size=16, cont=False, cont_src='', cosine=True, dataset='mnist', dir='TMP', epoch_scale=1, epochs=20000, epsilon=1e-05, finite_diff=False, hybrid=False, layers=0, lr=0.1, momentum=0.9, name='', nesterov=True, no_cuda=False, normalize_v=True, per_batch=False, print_freq_test=20, print_freq_train=100, save_model=False, seed=1, sigma=1.25, single=False, skips=True, start_epoch=0, strides=1, tD=False, tW=False, weight_decay=0.0005, widening=2)
1.25
==> Preparing data..
last_ch: 784, ch_out: 10 


In [119]:
X  = torch.randn([4,10])
Y  = torch.randn([4,3])

M1 = torch.matmul(X.unsqueeze(2),Y.unsqueeze(1)).sum(0)
M2 = torch.matmul(X.permute(1,0),Y)

M1-M2

tensor([[ 0.0000e+00,  0.0000e+00,  5.9605e-08],
        [ 1.1921e-07,  0.0000e+00,  1.1921e-07],
        [ 5.9605e-08,  4.7684e-07,  2.3842e-07],
        [ 0.0000e+00, -1.1921e-07,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.9605e-08, -5.9605e-08,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -1.1921e-07,  1.1921e-07],
        [ 0.0000e+00, -2.3842e-07,  0.0000e+00]])

In [123]:
(X >= 0).to(torch.float)

tensor([[0., 0., 1., 1., 1., 1., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0., 1., 0., 0., 1., 1.],
        [0., 0., 1., 1., 0., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]])

In [5]:
def L2_np(x):
    return x.norm().cpu().numpy()**2

In [129]:
X =torch.autograd.Variable(torch.randn([4,10]), requires_grad = True)
Y = torch.randint(10,[4])
L = F.cross_entropy(X, Y, reduction  = 'mean')
L.backward()
X.grad.shape



torch.Size([4, 10])

In [5]:
"""Train for one epoch on the training set"""
# switch to train mode
epoch = 0
model.train()
print('\nEpoch: %d' % epoch)
train_loss = 0
total = 0
correct = 0
epsilon = 1e-5
hybrid = True

optimizer.zero_grad()

for i, (input, target) in enumerate(dl_train):

    batch_size = input.shape[0]


    input = input.to(device)
    target = target.to(device)

    x1, x2, V = model.grad_v(input, hybrid = hybrid, epsilon = epsilon)

    loss1 =F.cross_entropy(x1, target, reduction  = 'none')
    loss2 =F.cross_entropy(x2, target, reduction  = 'none')    
    break


Epoch: 0


In [7]:

for linop in model.linops:
    linop.weight.grad = torch.zeros_like(linop.weight)
    linop.bias.grad = torch.zeros_like(linop.bias)


In [9]:


for b in range(batch_size):

    v_b_norm = np.sqrt(np.sum([ L2_np(_vw[b,:,:]) + L2_np(_vb)  for _vw,_vb in V.values()]))
    dF = (loss2[b]-loss1[b])/(v_b_norm*epsilon)
    for i,(vw,vb) in V.items():
        linop = model.linops[i]
        linop.weight.grad += dF*vw[b]
        linop.bias.grad += dF*vb

In [6]:
optimizer.step()


In [7]:
v_b_flat = torch.cat([_v[0].view(-1) for _v in V.values()])

NameError: name 'V' is not defined

In [8]:
v_b_flat.shape

torch.Size([5488])

In [10]:



def train_v(train_loader, model, args, optimizer, scheduler, epoch, device, writer=None,epsilon = 1e-5, hybrid = False):
    """Train for one epoch on the training set"""
    # switch to train mode
    
    model.train()
    print('\nEpoch: %d' % epoch)
    train_loss = 0
    total = 0
    correct = 0
    
    
    for i, (input, target) in tqdm(enumerate(train_loader), total = len(train_loader)):
        optimizer.zero_grad()

        batch_size = input.shape[0]

        
        input = input.to(device)
        target = target.to(device)

        x1, x2, V = model.grad_v(input, hybrid = hybrid,  epsilon = epsilon)
        
        
        
        for linop in model.linops:
            linop.weight.grad = torch.zeros_like(linop.weight)
            linop.bias.grad = torch.zeros_like(linop.bias)
            
        if hybrid:
            loss1 =F.cross_entropy(x1, target, reduction  = 'none')
            loss2 =F.cross_entropy(x2, target, reduction  = 'none')

            for b in range(batch_size):
                v_b_norm = np.sqrt(np.sum([ L2_np(_vw[b]) + L2_np(_vb)  for _vw,_vb in V.values()]))
                dF = (loss2[b]-loss1[b])/(v_b_norm*epsilon)
                for i,(vw,vb) in V.items():
                    linop = model.linops[i]
                    linop.weight.grad += dF*vw[b]
                    linop.bias.grad += dF*vb
                    
            loss = loss1.mean()
        else:
            loss1 =F.cross_entropy(x1, target, reduction  = 'mean')
            loss2 =F.cross_entropy(x2, target, reduction  = 'mean')
            
            v_norm = np.sqrt(np.sum([ L2_np(_vw) + L2_np(_vb)  for _vw,_vb in V.values()]))
            dF = (loss2-loss1)/(v_norm*epsilon)
            for i,(vw,vb) in V.items():
                linop = model.linops[i]
                linop.weight.grad += dF*vw
                linop.bias.grad += dF*vb
                
            loss = loss1


        pred = x1.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        train_loss += loss.item()

        total += input.size(0)
        optimizer.step()
        if args.cosine:
            scheduler.step()


    train_acc = (100. * correct / len(train_loader.dataset))
    train_loss = train_loss/(i+1)
    if writer is not None:
        writer.add_scalar('train/loss', train_loss, epoch)
        writer.add_scalar('train/acc', train_acc, epoch)


    return train_loss, train_acc


In [11]:

train_v(dl_train, model, args, optimizer, scheduler, 0, device)


Epoch: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:20<00:00, 92.94it/s]


(1078.9549351334572, 13.476666666666667)

In [16]:
loglr = round(np.log(args.lr)/np.log(10))
lead = round(args.lr / (10**loglr))

lr = f"{lead}E{loglr}"
lr

'3E-5'